In [ ]:
!pip install python-igraph
!pip install cairocffi

In [ ]:
import ast
import math
import random
import numpy as np
import pandas as pd
import igraph as ig
import networkx as nx
import cairocffi as cairo
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("school_salathe.csv", names=["Time","ID1","ID2","count"],header=None, sep=' ')
data

In [ ]:
test_df = data.copy()

In [ ]:
net = np.loadtxt('school_salathe.csv').astype(int) # read network
print(net[:10,:]) # print first contacts

In [ ]:
# 1. Total number of people in the data
# 2. Time unite of data
# 3. Graph analysis
# 4. Finding probable infected individuals(as seed)
# 5. simulation
# 6. correlation between graph results and simulation

# 1. Total number of people in the data

In [ ]:
print("Total length of people(students(655),teachers(73), and staff(55),other(5)) :" ,len(np.unique( test_df[["ID1","ID2"]] ) ))

# 2. Time unite of data

Each time step corresponds to roughly 10 minutes. The weight corresponds to the number of contacts registered within the 10 minutes, where each contact can last ~20 sec



# 3. Graph analysis

In [ ]:
len(np.unique(data[data.Time==0][['ID1','ID2']]))

In [ ]:
# we dont need any average analysis for this network
# we should gatheer all graphs' properties and compare different individuals for disease simulation

In [ ]:
snapshots = [ l for isx, l in data.groupby("Time")]


Properties = []
for net in snapshots:
  nodes = np.unique(net[["ID1","ID2"]])
  G = nx.Graph()
  G.add_nodes_from(nodes)
  G.add_weighted_edges_from(np.array(net[["ID1","ID2","count"]]))
  g_prop = ig.Graph.from_networkx(G)
  Properties.append({"snapshot": [net.Time.iloc[0]]*len(nodes), "Density": [(2 * len(G.edges()) )/(788 * 787)]*len(nodes),
                     "Nodes(ID)": nodes, "Degree" : g_prop.degree(), "PageRank": g_prop.pagerank(directed=False),
                     "Betweenness": g_prop.betweenness(directed=False)})

In [ ]:
df = pd.concat((pd.DataFrame(test) for test in Properties))
df

# 4. Finding probable infected individuals(as seed)

In [ ]:
degree_seeds   = list(df.sort_values(by='Degree',ascending=False).head(10)['Nodes(ID)'])
page_seeds     = list(df.sort_values(by='PageRank',ascending=False).head(10)['Nodes(ID)'])
between_seeds  = list(df.sort_values(by='Betweenness',ascending=False).head(10)['Nodes(ID)'])
var_seeds      = list(df.groupby('Nodes(ID)').Degree.describe().sort_values(by='std',ascending=False).head(10).index)
selected_seeds = np.unique(degree_seeds+page_seeds+between_seeds+var_seeds)
selected_seeds

# Finding Transimissibility with having a R0 less than 1:

In [ ]:
def simulate_SIR( tr, network, N0, seed):
    '''
    This functions simulates a SIR process on a temporal network
    
    Arguments:
    
    1) t_max: number of simulation time steps; if t_max is larger than the duration of the network, 
    periodic boundary conditions are applied. The simulation may stop earlier if the epidemics dies out.
    
    2) tr: per-contact probability to transmit the disease from an infected individual to a susceptible one 
    during a single time step 
    
    3) rec: probability to become recovered during a single time step
    
    4) N0: number of initially infected individuals
    
    Output:
    
    a list containing the prevalence during each time step
    
    '''
    
    # T = np.amax(network[:,0]) # get network duration (the period)
    node_labels = np.unique( network[:, 1:3])  # get nodes' labels
    
    state = {i: 'S' for i in node_labels} # set all nodes as susceptibles ('S')
    
    #===== Seed the infection =====#
    
    # first_nodes = np.unique( network[ network[:,0] == 0 ][:, 1:3]) # select all nodes appearing in the first snapshot
    
    # Choose N0 individuals at random among the nodes that appear in the first snapshot 
    # if N0 < len(first_nodes):
    #     seeds = np.random.choice(first_nodes, size = N0, replace = False)
    # else:
    #     seeds = first_nodes
    
    # Set the chosen nodes to infected (I)
    # for seed in seeds:
    state[seed] = 'I'
        
    prevalence = [N0] # This list will store the results
    
    #===== Start the simulation =====#
    
    # for t in range(t_max):
    snapshot = network[ network[:,0] == 0 ] # select the snapshot (use t mod T in order to use periodic boundary conditions)    
    new_infected = [] # This list will store the nodes that will become infected during this time step
    
    # loop over contacts in the current snapshot
    for edge in snapshot[:,]:
        s1 = state[ edge[1] ]
        s2 = state[ edge[2] ]
        
        # check if the contact is between a susceptible and an infected node
        if ( (s1 == 'S') and (s2 == 'I') ) or  ( (s2 == 'S') and (s1 == 'I') ) :
            
            if s1 == 'S':
                target_node = edge[1]
            else:  
                target_node = edge[2]
                
            # check if infection occurs with probability given by the transmissibility ('tr')
            # if infection occurs, do not set the susceptible node infected straight away
            # but store this information in 'new_infected'.
            # Also check that the susceptible node has not been infected yet!
            if target_node not in new_infected:
                if np.random.random() < tr:
                    new_infected.append( target_node )

    return new_infected  

## seed: 287

In [ ]:
transmissibility = np.linspace(0.01, 0.025, num=100)
N0 = 1
np.random.seed(123)

result = []
for trs in transmissibility:
  for i in range(100):
    R0 = simulate_SIR(tr = trs, network = net, N0 = N0, seed= 287)
    result.append({"seed": 287, "tr":trs, 'R0': len(R0)})

In [ ]:
test = pd.DataFrame(result)

In [ ]:
mean_R0 = test.groupby('tr').R0.describe()
mean_R0

In [ ]:
import plotly.express as px
px.line(mean_R0 ,x=mean_R0.index, y='mean',title = "R0 versus transmissibility(seed:287)")

## sedd:365

In [ ]:
transmissibility = np.linspace(0.05, 0.06, num=100)
N0 = 1
np.random.seed(123)

result = []
for trs in transmissibility:
  for i in range(30):
    R0 = simulate_SIR(tr = trs, network = net, N0 = N0, seed= 365)
    result.append({"seed": 365, "tr":trs, 'R0': len(R0)})

In [ ]:
test = pd.DataFrame(result)

In [ ]:
mean_R0 = test.groupby('tr').R0.describe()
mean_R0

In [ ]:
import plotly.express as px
px.line(mean_R0 ,x=mean_R0.index, y='mean',title = "R0 versus transmissibility(seed:365)")

# 5. SIR Simulation

In [ ]:
def simulate_SIR(t_max, tr, rec, network, N0, seed):
    '''
    This functions simulates a SIR process on a temporal network
    
    Arguments:
    
    1) t_max: number of simulation time steps; if t_max is larger than the duration of the network, 
    periodic boundary conditions are applied. The simulation may stop earlier if the epidemics dies out.
    
    2) tr: per-contact probability to transmit the disease from an infected individual to a susceptible one 
    during a single time step 
    
    3) rec: probability to become recovered during a single time step
    
    4) N0: number of initially infected individuals
    
    Output:
    
    a list containing the prevalence during each time step
    
    '''
    
    T = np.amax(network[:,0]) # get network duration (the period)
    node_labels = np.unique( network[:, 1:3])  # get nodes' labels
    
    state = {i: 'S' for i in node_labels} # set all nodes as susceptibles ('S')
    
    #===== Seed the infection =====#
    
    # first_nodes = np.unique( network[ network[:,0] == 0 ][:, 1:3]) # select all nodes appearing in the first snapshot
    
    # Choose N0 individuals at random among the nodes that appear in the first snapshot 
    # if N0 < len(first_nodes):
    #     seeds = np.random.choice(first_nodes, size = N0, replace = False)
    # else:
    #     seeds = first_nodes
    
    # Set the chosen nodes to infected (I)
    # for seed in seeds:
    state[seed] = 'I'
        
    prevalence = [N0] # This list will store the results
    
    #===== Start the simulation =====#
    
    for t in range(t_max):
        snapshot = network[ network[:,0] == t % T ] # select the snapshot (use t mod T in order to use periodic boundary conditions)    
        new_infected = [] # This list will store the nodes that will become infected during this time step
        
        # loop over contacts in the current snapshot
        for edge in snapshot[:,]:
            s1 = state[ edge[1] ]
            s2 = state[ edge[2] ]
            
            # check if the contact is between a susceptible and an infected node
            if ( (s1 == 'S') and (s2 == 'I') ) or  ( (s2 == 'S') and (s1 == 'I') ) :
                
                if s1 == 'S':
                    target_node = edge[1]
                else:  
                    target_node = edge[2]
                    
                # check if infection occurs with probability given by the transmissibility ('tr')
                # if infection occurs, do not set the susceptible node infected straight away
                # but store this information in 'new_infected'.
                # Also check that the susceptible node has not been infected yet!
                if target_node not in new_infected:
                    if np.random.random() < tr:
                        new_infected.append( target_node )
              
        # loop over nodes; if a node is infected, it recovers with probability given by 'rec' and its
        # status is set to 'R'
        for node, s in state.items():
            if s == 'I':
                if np.random.random() < rec:
                    state[node] = 'R'
        
        # finally update the status of nodes that have been successfully infected
        for node in new_infected:
            state[node] = 'I'
          
        # Compute the prevalence and store it
        prev = len([node for node, s in state.items() if s == 'I'])
        prevalence.append({'prev':prev,'prev_id':new_infected})
        
        # if there no infected halt the simulation
        if (prev == 0) : # or (t > t_max)
            break
            
    return prevalence

## Simulation on selected seeds(all)

In [ ]:
t_max = 6*24*100 # maximum simulation time ( 10(day)* 24(hour)* 6 (10 minutes) )
transmissibility = 0.00016 # infection probability 0.016
recovery = 0.001 # recovery probability
N0 = 1 # initial seeds

results = []
for seed in selected_seeds:
  np.random.seed(123)
  prevalence = simulate_SIR(t_max, tr = transmissibility, rec = recovery, network = net, N0 = N0, seed= seed)
  total = len(np.unique(sum([a_dict['prev_id'] for a_dict in prevalence[1:]],[])))+1
  prev = [a_dict['prev'] for a_dict in prevalence[1:]]
  results.append({"seed":seed,'total':total , 'max_prev': max(prev), 'prevalence': prev})

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_pickle(f"results_all(salathe).pkl")
results_df.sort_values(by='total',ascending=False)

## Simulation on selected seeds(Degree_std)

In [ ]:
t_max = 6*24*100 # maximum simulation time ( 10(day)* 24(hour)* 6 (10 minutes) )
transmissibility = 0.00016 # infection probability 0.016
recovery = 0.001 # recovery probability
N0 = 1 # initial seeds

results = []
for seed in selected_seeds:
  np.random.seed(123)
  prevalence = simulate_SIR(t_max, tr = transmissibility, rec = recovery, network = net, N0 = N0, seed= seed)
  total = len(np.unique(sum([a_dict['prev_id'] for a_dict in prevalence[1:]],[])))+1
  prev = [a_dict['prev'] for a_dict in prevalence[1:]]
  results.append({"seed":seed,'total':total , 'max_prev': max(prev), 'prevalence': prev})

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_pickle(f"results_Degree_std(salathe).pkl")
results_df.sort_values(by='total',ascending=False)

In [ ]:
for i in range(10):
  plt.plot(results_df['prevalence'].iloc[i])
  plt.show()

## Simulation on selected seeds(Degree)

In [ ]:
t_max = 6*24*100 # maximum simulation time ( 10(day)* 24(hour)* 6 (10 minutes) )
transmissibility = 0.00016 # infection probability 0.016
recovery = 0.001 # recovery probability
N0 = 1 # initial seeds

selected_seeds = df[(df.snapshot==0)].sort_values(by='Degree',ascending=False).head(10)['Nodes(ID)']
selected_seeds = list(selected_seeds)
selected_seeds

results = []
for seed in selected_seeds:
  np.random.seed(123)
  prevalence = simulate_SIR(t_max, tr = transmissibility, rec = recovery, network = net, N0 = N0, seed= seed)
  total = len(np.unique(sum([a_dict['prev_id'] for a_dict in prevalence[1:]],[])))+1
  prev = [a_dict['prev'] for a_dict in prevalence[1:]]
  results.append({"seed":seed,'total':total , 'max_prev': max(prev), 'prevalence': prev})

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_pickle(f"results_Degree(salathe).pkl")
results_df.sort_values(by='total',ascending=False)

## Simulation on selected seeds(PageRank)

In [ ]:
t_max = 6*24*100 # maximum simulation time ( 10(day)* 24(hour)* 6 (10 minutes) )
transmissibility = 0.00016 # infection probability 0.016
recovery = 0.001 # recovery probability
N0 = 1 # initial seeds

selected_seeds = df[(df.snapshot==0)].sort_values(by='PageRank',ascending=False).head(10)['Nodes(ID)']
selected_seeds = list(selected_seeds)
selected_seeds

results = []
for seed in selected_seeds:
  np.random.seed(123)
  prevalence = simulate_SIR(t_max, tr = transmissibility, rec = recovery, network = net, N0 = N0, seed= seed)
  total = len(np.unique(sum([a_dict['prev_id'] for a_dict in prevalence[1:]],[])))+1
  prev = [a_dict['prev'] for a_dict in prevalence[1:]]
  results.append({"seed":seed,'total':total , 'max_prev': max(prev), 'prevalence': prev})

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_pickle(f"results_pagerank(salathe).pkl")
results_df.sort_values(by='total',ascending=False)

## Simulation on selected seeds(Betweenness)

In [ ]:
t_max = 6*24*100 # maximum simulation time ( 10(day)* 24(hour)* 6 (10 minutes) )
transmissibility = 0.00016 # infection probability 0.016
recovery = 0.001 # recovery probability
N0 = 1 # initial seeds

selected_seeds = df[(df.snapshot==0)].sort_values(by='Betweenness',ascending=False).head(10)['Nodes(ID)']
selected_seeds = list(selected_seeds)
selected_seeds

results = []
for seed in selected_seeds:
  np.random.seed(123)
  prevalence = simulate_SIR(t_max, tr = transmissibility, rec = recovery, network = net, N0 = N0, seed= seed)
  total = len(np.unique(sum([a_dict['prev_id'] for a_dict in prevalence[1:]],[])))+1
  prev = [a_dict['prev'] for a_dict in prevalence[1:]]
  results.append({"seed":seed,'total':total , 'max_prev': max(prev), 'prevalence': prev})

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_pickle(f"results_betweenness(salathe).pkl")